In [1]:
# system imports
import os
import time

In [2]:
# model imports
# All torch related imports
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import transforms
from torch import nn, optim
import torchvision

In [3]:
from torch.nn import Softmax

In [4]:
# visualization imports
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import numpy as np

In [5]:
# metrics imports
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report

In [6]:
from tqdm.notebook import tqdm as tq

In [7]:
IMAGE_DIMS = 224

In [8]:
# pytorch device configurations
BATCH_SIZE = 8
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
TEST = pd.read_pickle('./sync_pickles/testing_set.pkl')

In [10]:
TEST

,labels,images
0,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
1,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
2,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
3,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
4,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
...,...,...
8515,170,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
8516,170,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
8517,170,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
8518,170,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."


In [11]:
class MetricsCalculator:
    def __init__(self):

        self.accuracy = 0
        self.loss = 0
        self.macro_f1_score = 0
        self.accuracy_history = []
        self.loss_history = []
        self.macro_f1_history = []

        self.all_preds = torch.tensor([])
        self.top_preds = torch.tensor([])
        self.truth = torch.tensor([])

    def compute_scores(self, preds_tensor, labels_tensor, loss):
        softmax = Softmax(dim=1)
        preds_tensor = softmax(preds_tensor)
        
        values, max_indices = torch.max(preds_tensor, dim=1)

        preds = max_indices.detach().cpu().numpy()
        truth = labels_tensor.detach().cpu().numpy()

        accuracy = accuracy_score(truth, preds)
        macro_f1 = f1_score(truth, preds, average="macro")
        
        self.accuracy += accuracy
        self.macro_f1_score += macro_f1
        self.loss += loss.detach().cpu()
        
        return accuracy, macro_f1

    def save_predictions(self, preds, labels, index):
        softmax = Softmax(dim=1)
        
        preds = preds.detach().cpu()
        labels = labels.detach().cpu()
        
        preds_tensor = softmax(preds)
        values, max_indices = torch.max(preds_tensor, dim=1)
        
        results = torch.topk(preds_tensor, 3)
        self.all_preds = torch.cat((self.all_preds, max_indices), dim=0)
        #self.top_preds = torch.cat((self.top_preds, results), dim=0)
        print(type(results))
        self.truth = torch.cat((self.truth, labels), dim=0)

In [12]:
class LeanIsolatedCharacterDataset(Dataset):
    def __init__(self, x, y, transforms=None):
        ### labels
        self.labels = y.to_numpy()

        ### loading dataset into memory
        self.images = x.to_numpy()

        ### transformations to apply on images
        self.transforms = transforms

    def __getitem__(self, index):
        # convert labels to tensor
        label = torch.tensor(self.labels[index])
        # load single image from list of all preloaded images
        image = self.images[index]
        if self.transforms:
            ## apply transforms
            image = self.transforms(image)
            image = image.float()
        label = label.long()
        return image, label, index

    def __len__(self):
        return len(self.images)

In [13]:
DATA_NORMALIZER = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),transforms.Resize((IMAGE_DIMS,IMAGE_DIMS))])

In [14]:
TESTER = LeanIsolatedCharacterDataset(TEST.images, TEST.labels, DATA_NORMALIZER)

In [15]:
TESTING_LOADER = DataLoader( dataset=TESTER, batch_size=BATCH_SIZE , shuffle=False )

In [16]:
class block(nn.Module):
    def __init__(
        self, in_channels, intermediate_channels, identity_downsample=None, stride=1
    ):
        super(block, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(
            in_channels, intermediate_channels, kernel_size=1, stride=1, padding=0
        )
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        self.conv2 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
        )
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        self.conv3 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels * self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
        )
        self.bn3 = nn.BatchNorm2d(intermediate_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    # Identity block
    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)
        # x = self.relu(x) #custom

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x

In [17]:
class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Essentially the entire ResNet architecture are in these 4 lines below
        self.layer1 = self._make_layer(
            block, layers[0], intermediate_channels=64, stride=1
        )
        self.layer2 = self._make_layer(
            block, layers[1], intermediate_channels=128, stride=2
        )
        self.layer3 = self._make_layer(
            block, layers[2], intermediate_channels=256, stride=2
        )
        self.layer4 = self._make_layer(
            block, layers[3], intermediate_channels=512, stride=2
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x

    def _make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []

        # Either if we half the input space for ex, 56x56 -> 28x28 (stride=2), or channels change
        # we need to adapt the Identity (skip connection) so it will be able to be added
        # to the layer that's ahead
        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    intermediate_channels * 4,
                    kernel_size=1,
                    stride=stride,
                ),
                nn.BatchNorm2d(intermediate_channels * 4),
            )

        layers.append(
            block(self.in_channels, intermediate_channels, identity_downsample, stride)
        )

        # The expansion size is always 4 for ResNet 50,101,152
        self.in_channels = intermediate_channels * 4

        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)

In [18]:
def ResNet50(img_channel=3, num_classes=171):
    return ResNet(block, [2, 3, 5, 2], img_channel, num_classes)
net1 = ResNet50()

In [19]:
MODEL = torchvision.models.resnext50_32x4d(pretrained= False, progress = True)

In [20]:
MODEL.fc = nn.Linear(2048, 171)

In [21]:
MODEL.fc

Linear(in_features=2048, out_features=171, bias=True)

In [22]:
net = MODEL

In [23]:
net.load_state_dict(torch.load('./completed/model_2/custom_resnet_50_minimum_validation_f1_score_240x240_0_1model.pth'))

<All keys matched successfully>

In [24]:
optimizer = optim.Adam(net.parameters(), lr=0.07)  # learning rate
# defining the loss function
criterion = nn.CrossEntropyLoss()  # reduction='none'
net = net.to(DEVICE)
criterion = criterion.to(DEVICE)

In [25]:
def test(model,data_loader):
    model.eval()
    print('testing')
    metrics = MetricsCalculator()
    
    for i,items in tq(enumerate(data_loader, 0)):
        inputs, labels, indexes = items
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        with torch.no_grad():
            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            metrics.compute_scores(preds_tensor=outputs, labels_tensor=labels, loss=loss)
            
            # saving predictions 
            metrics.save_predictions(outputs, labels, indexes)
    
    print(len(data_loader))
    
    metrics.accuracy = metrics.accuracy /len(data_loader)
    metrics.loss = metrics.loss /len(data_loader)
    metrics.macro_f1_score = metrics.macro_f1_score/ len(data_loader)
    
    return metrics            

In [26]:
results = test(net,TESTING_LOADER)

testing


<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.

<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.

<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.

<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.

<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.

In [27]:
cm = confusion_matrix(results.truth.numpy().astype(int), results.all_preds.numpy().astype(int))

In [28]:
results.all_preds.shape

torch.Size([8520])

In [29]:
len(cm)

171

In [30]:
results.accuracy

0.9657276995305164

In [31]:
results.macro_f1_score

0.8935361699680959

In [32]:
results.truth.numpy()

array([  0.,   0.,   0., ..., 170., 170., 170.], dtype=float32)

In [33]:
results.all_preds.numpy()

array([  0.,   0.,   0., ..., 170., 170., 170.], dtype=float32)

In [34]:
LABEL = [x for x in range(171)]

In [35]:
report = classification_report(results.truth.numpy(), results.all_preds.numpy(), labels=LABEL,output_dict=True) #output_dict=True

In [36]:
cm

array([[52,  0,  0, ...,  0,  0,  0],
       [ 0, 42,  0, ...,  0,  0,  0],
       [ 0,  0, 48, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ..., 54,  0,  0],
       [ 0,  0,  0, ...,  0, 54,  0],
       [ 0,  0,  0, ...,  0,  0, 48]], dtype=int64)

In [37]:
from sklearn.metrics import plot_confusion_matrix as plotter

In [38]:
cm_df = pd.DataFrame(cm)

In [39]:
cm_df

,0,1,2,3,4,5,6,7,8,9,...,161,162,163,164,165,166,167,168,169,170
0,52,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,42,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,48,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,53,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,51,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,54,0,0,0,0
167,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,49,0,0,0
168,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,54,0,0
169,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,54,0


In [43]:
cm_df.to_html('./results/custom_restnet_28x28/pretrained_restnet_224x224.html')

In [44]:
len(report)

174

In [45]:
report

{'0': {'precision': 0.9811320754716981,
  'recall': 1.0,
  'f1-score': 0.9904761904761905,
  'support': 52},
 '1': {'precision': 1.0,
  'recall': 0.9767441860465116,
  'f1-score': 0.988235294117647,
  'support': 43},
 '2': {'precision': 0.8888888888888888,
  'recall': 0.9795918367346939,
  'f1-score': 0.9320388349514563,
  'support': 49},
 '3': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 53},
 '4': {'precision': 0.9622641509433962,
  'recall': 1.0,
  'f1-score': 0.9807692307692307,
  'support': 51},
 '5': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 49},
 '6': {'precision': 0.9574468085106383,
  'recall': 0.8823529411764706,
  'f1-score': 0.9183673469387754,
  'support': 51},
 '7': {'precision': 0.9411764705882353,
  'recall': 0.9795918367346939,
  'f1-score': 0.96,
  'support': 49},
 '8': {'precision': 0.9666666666666667,
  'recall': 1.0,
  'f1-score': 0.983050847457627,
  'support': 58},
 '9': {'precision': 0.9807692307692307,
  'recall': 0.92727272

In [46]:
pd.DataFrame(report).transpose()

,precision,recall,f1-score,support
0,0.981132,1.000000,0.990476,52.000000
1,1.000000,0.976744,0.988235,43.000000
2,0.888889,0.979592,0.932039,49.000000
3,1.000000,1.000000,1.000000,53.000000
4,0.962264,1.000000,0.980769,51.000000
...,...,...,...,...
169,1.000000,1.000000,1.000000,54.000000
170,0.979592,1.000000,0.989691,48.000000
accuracy,0.965728,0.965728,0.965728,0.965728
macro avg,0.967481,0.965159,0.965000,8520.000000


In [47]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [49]:
pd.DataFrame(report).transpose()

,precision,recall,f1-score,support
0,0.981132,1.000000,0.990476,52.000000
1,1.000000,0.976744,0.988235,43.000000
2,0.888889,0.979592,0.932039,49.000000
3,1.000000,1.000000,1.000000,53.000000
4,0.962264,1.000000,0.980769,51.000000
5,1.000000,1.000000,1.000000,49.000000
6,0.957447,0.882353,0.918367,51.000000
7,0.941176,0.979592,0.960000,49.000000
8,0.966667,1.000000,0.983051,58.000000
9,0.980769,0.927273,0.953271,55.000000


In [51]:
cm_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170
0,52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,42,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,53,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,51,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,49,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,48,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,58,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,1,0,0,51,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0